In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import json

In [9]:
#create a sql database in memory
engine = create_engine('sqlite://', echo=False)

#process data and get the result
from MSHA_transform import filtered_data as df

In [11]:
#the processed data, ready to be loaded onto a sql database
df.head(2)

,MINE_ID,CURRENT_MINE_NAME,COAL_METAL_IND,CURRENT_MINE_TYPE,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_ID,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,CURRENT_OPERATOR_NAME,...,PRIMARY_CANVASS_CD,PRIMARY_CANVASS,SECONDARY_CANVASS_CD,SECONDARY_CANVASS,PORTABLE_OPERATION,PORTABLE_FIPS_ST_CD,LONGITUDE,LATITUDE,DIRECTIONS_TO_MINE,NEAREST_TOWN
0,100003,O'Neal Quarry & Mill,M,Surface,Active,01/22/1979,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",...,6.0,Stone,6.0,Stone,N,NaN,85.753333,38.256389,Approx 7 miles south of the city of Alabaster ...,Calera
1,100004,Brierfield Quarry,M,Surface,Active,03/04/2003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",...,6.0,Stone,6.0,Stone,N,NaN,86.963333,33.038056,"I-65 SOUTH EXIT US 31 TO CALERA, TURN RIGHT ON...",Brierfield


In [12]:

#make a sql table called MSHA based on the filtered data
df.to_sql('MSHA', engine, if_exists='replace', index=False)

#write sql statement here to retrieve data
#can also use other built-in functions instead of straight sql statements

#a sample query (find suppliers in Texas for Construction Sand and Gravel)
results = engine.execute("SELECT * FROM MSHA WHERE STATE = 'TX' AND PRIMARY_SIC = 'Construction Sand and Gravel'")

#turn results into a dataframe (should be json-array or object if using PHP/JS)
final_results = pd.DataFrame(results, columns=df.columns)

In [13]:
len(final_results)

239

In [14]:
#mapping library (Python wrapper around Leaflet.js)

import folium
from folium.plugins import MarkerCluster

In [15]:
#some extra processing on the client-side

final_results.fillna('no record', inplace=True)

In [16]:
#mapping script (probably in JS to have more functionalities)

from string import Template

tooltip_template = Template('Name: ${name}<br>Controller: ${controller}<br>Operator: ${operator}<br><br>Primary Product: ${primary_prod}<br>Secondary Product: ${secondary_prod}<br>Primary Canvass: ${primary_canvass}<br>Secondary Canvass: ${secondary_canvass}<br>Nearest Town: ${nearest_town}')

markers = []
for r in final_results.index:
    row = final_results.loc[r, :]
    #need to make LONGITUDE negative
    if row['LATITUDE'] == 'no record' or row['LONGITUDE'] == 'no record':
        continue
    location = [row['LATITUDE'], -row['LONGITUDE']]
    
    tooltip_content = tooltip_template.substitute({
        'name': row['CURRENT_MINE_NAME'],
        'controller': row['CURRENT_CONTROLLER_NAME'],
        'operator': row['CURRENT_OPERATOR_NAME'],
        'primary_prod': row['PRIMARY_SIC'],
        'secondary_prod': row['SECONDARY_SIC'],
        'primary_canvass': row['PRIMARY_CANVASS'],
        'secondary_canvass': row['SECONDARY_CANVASS'],
        'nearest_town': row['NEAREST_TOWN']
    })
    popup = folium.Popup(str(row['DIRECTIONS_TO_MINE']))
    tooltip = folium.Tooltip(tooltip_content)
    markers.append(folium.Marker(location=location, popup=popup, tooltip=tooltip))


In [17]:
map_center = [31, -100] #using Texas coords as the center
m = folium.Map(location=map_center, zoom_start = 4)
mcg = folium.plugins.MarkerCluster()
for marker in markers:
    marker.add_to(mcg)
mcg.add_to(m)
m

In [18]:
m.save('./MSHA_demo.html')